In [1]:
import random
import sys

import pyomo.kernel as pmo


import numpy as np


# Set to True to show 3d plots
show_plots = False

def f(x, y, package=pmo):
    return (-20 * package.exp(
                -2.0 * package.sqrt(0.5 * (x**2 + y**2))) -
            package.exp(
                0.5 * (package.cos(2*np.pi*x) + \
                       package.cos(2*np.pi*y))) + \
            np.e + 20.0)

def g(x, y, package=pmo):
    return (x-3)**2 + (y-1)**2

m = pmo.block()
m.x = pmo.variable(lb=-5, ub=5)
m.y = pmo.variable(lb=-5, ub=5)
m.z = pmo.variable()
m.obj = pmo.objective(m.z)

m.real = pmo.block()
m.real.f = pmo.constraint(m.z >= f(m.x, m.y))
m.real.g = pmo.constraint(m.z == g(m.x, m.y))

#
# Approximate f and g using piecewise-linear functions
#

m.approx = pmo.block()

tri = pmo.piecewise_util.generate_delaunay([m.x, m.y], num=25)
pw_xarray, pw_yarray = np.transpose(tri.points)

fvals = f(pw_xarray, pw_yarray, package=np)
pw_f = pmo.piecewise_nd(tri,
                        fvals,
                        input=[m.x,m.y],
                        output=m.z,
                        bound='lb')
m.approx.pw_f = pw_f

gvals = g(pw_xarray, pw_yarray, package=np)
pw_g = pmo.piecewise_nd(tri,
                        gvals,
                        input=[m.x,m.y],
                        output=m.z,
                        bound='eq')
m.approx.pw_g = pw_g

#
# Solve the approximate model to generate a warmstart for
# the real model
#

m.real.deactivate()
m.approx.activate()
glpk = pmo.SolverFactory("gurobi")
status = glpk.solve(m)


print("Approximate f value at MIP solution: %s"
      % (pw_f((m.x.value, m.y.value))))
print("Approximate g value at MIP solution: %s"
      % (pw_g((m.x.value, m.y.value))))
print("Real f value at MIP solution: %s"
      % (f(m.x.value, m.y.value)))
print("Real g value at MIP solution: %s"
      % (g(m.x.value, m.y.value)))

#
# Solve the real nonlinear model using a local solver
#

m.real.activate()
m.approx.deactivate()
ipopt = pmo.SolverFactory("ipopt")
status = ipopt.solve(m)
assert str(status.solver.status) == "ok"
assert str(status.solver.termination_condition) == "optimal"
print("Real f value at NL solution: %s"
      % (f(m.x.value, m.y.value)))
print("Real g value at NL solution: %s"
      % (f(m.x.value, m.y.value)))


    #
    # Plot the approximation of f
    #
#
# Plot the approximation of g
#



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information